In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os, sys
import toml
config = toml.load('../../../../configuration/input_configuration.toml')
%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
working_dir = r'../../../../outputs/agg/dash'
# pd.options.display.float_format = '{:0,.0f}'.format
format_percent = "{:,.2%}".format

In [3]:
def load_data(fname):
    """ open and join model and survey data """

    df_model = pd.read_csv(os.path.join(working_dir,fname+'.csv'))
    df_model['source'] = 'model'
    df_survey = pd.read_csv(os.path.join(working_dir,'survey',fname+'.csv'))
    df_survey['source'] = 'survey'
    df = df_model.append(df_survey)

    return df

In [4]:
parcel_geog = pd.read_sql_table('parcel_'+config['base_year']+'_geography', 'sqlite:///../../../../inputs/db/'+config['db_name'],
        columns=['CountyName','ParcelID'])

In [5]:
def dist_by_mode(purpose=None):
    df = load_data('tour_distance')
    # Trim any unreasonable survey distances off by taking max distance as max modeled bin value
    max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
    df = df.loc[df['tautodist_bin'] <= max_model_dist]
    df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
    df_dist = pd.DataFrame()
    for source in ['model','survey']:
        if purpose is not None:
            _df = df[(df['source'] == source) & (df['pdpurp'] == purpose)]
        else:
            _df = df[(df['source'] == source)]
        df1 = _df.groupby('tmodetp').sum()[['wt_autodist']].reset_index()
        df2 = _df.groupby('tmodetp').sum()[['toexpfac']].reset_index ()
        _df = df1.merge(df2, on='tmodetp')
        _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
        _df['source'] = source
        df_dist = df_dist.append(_df)
    df = df_dist.pivot_table(index='tmodetp', columns='source', aggfunc='first', values='commute_distance')
    df.rename(columns={'tmodetp': 'Tour Mode'}, inplace=True)
    for col in ['School Bus',' Park']:
        if (col in df.index) & (purpose != 'School'):
            df.drop(col, axis=0, inplace=True)
        if (col != 'School Bus') & (col in df.index):
            df.drop(col, axis=0, inplace=True)
    
    return df

### Total

In [6]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,3.868672
survey,3.927769


### By Purpose

In [7]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
# df.drop('Change Mode Inserted Purpose', axis=0, inplace=True)
df

source,model,survey
pdpurp,,
Escort,7.180035,4.359001
Meal,5.918669,3.757344
Personal Business,6.906479,6.631983
School,3.868672,3.927769
Shop,4.565767,4.274982
Social,4.525877,5.629700
Work,11.598226,11.402478


# Tour Distance
### All Tours

In [8]:
dist_by_mode()

source,model,survey
tmodetp,,
Bike,3.535137,2.358237
HOV2,7.115245,5.932619
HOV3+,6.881907,6.832261
Park,16.067399,16.356694
SOV,8.577349,8.316768
TNC,2.320240,6.928344
Transit,8.888232,9.180212
Walk,0.972854,0.844283


### Average Work Tour Distance By Mode

In [9]:
dist_by_mode('Work')

source,model,survey
tmodetp,,
Bike,5.524815,3.219214
HOV2,11.662787,12.056786
HOV3+,12.107762,14.218483
Park,16.067399,16.524274
SOV,12.271868,11.523192
TNC,5.343801,8.997572
Transit,12.768083,10.923398
Walk,0.761746,1.104276


### Average School Tour Distance by Mode

In [10]:
dist_by_mode('School')

source,model,survey
tmodetp,,
Bike,2.668292,2.332524
HOV2,3.622195,3.061157
HOV3+,3.672618,4.202338
SOV,8.522732,8.040764
School Bus,3.173636,2.901791
TNC,2.457853,NaN
Transit,6.997357,8.549150
Walk,1.116296,0.829758


### Average Personal Business Tour Distance by Mode

In [11]:
dist_by_mode('Personal Business')

source,model,survey
tmodetp,,
Bike,3.274935,2.328799
HOV2,8.063055,7.062631
HOV3+,7.885211,9.720407
Park,NaN,22.182039
SOV,6.826884,4.687277
TNC,2.255603,6.124703
Transit,7.037309,13.585403
Walk,1.207712,1.534727


### Average Shopping Tour Distance by Mode

In [12]:
dist_by_mode('Shop')

source,model,survey
tmodetp,,
Bike,2.380708,0.798839
HOV2,5.320172,5.307668
HOV3+,5.371932,5.059006
SOV,4.631032,4.123578
TNC,1.602992,NaN
Transit,4.046971,3.529301
Walk,1.095107,0.755967


### Average Meal Tour Distance by Mode

In [13]:
dist_by_mode('Meal')

source,model,survey
tmodetp,,
Bike,1.606443,1.149049
HOV2,8.759313,3.871146
HOV3+,9.404576,4.485626
Park,NaN,14.389210
SOV,5.547977,3.677246
TNC,1.475845,1.500000
Transit,6.843750,3.778609
Walk,0.706018,0.823848


### Average Social Tour Distance by Mode

In [14]:
dist_by_mode('Social')

source,model,survey
tmodetp,,
Bike,2.588317,2.348585
HOV2,6.090767,5.623100
HOV3+,6.041473,7.486065
Park,NaN,16.082474
SOV,5.405942,6.824962
TNC,1.636076,4.949222
Transit,5.627068,9.924383
Walk,0.973979,0.801136


### Average Escort Tour Distance by Mode

In [15]:
dist_by_mode('Escort')

source,model,survey
tmodetp,,
Bike,13.203470,1.370234
HOV2,6.714988,4.012821
HOV3+,7.297905,5.144569
SOV,21.434321,6.211452
TNC,1.674847,1.500000
Transit,2.666667,6.946770
Walk,1.053103,0.790155


Survey Sample Size Reference by Mode and Purpose

In [16]:
# Survey Sample size
df_tour_survey = pd.read_csv(r'../../../../inputs/base_year/survey/_tour.tsv', delim_whitespace=True)

In [17]:
df_tour_survey.pivot_table(index='tmodetp',columns='pdpurp', values='toexpfac', aggfunc='count')

pdpurp,1,2,3,4,5,6,7
tmodetp,,,,,,,
1,236.0,73.0,115.0,77.0,300.0,267.0,1078.0
2,121.0,21.0,17.0,6.0,16.0,22.0,87.0
3,1396.0,45.0,70.0,358.0,580.0,228.0,832.0
4,263.0,96.0,368.0,160.0,302.0,211.0,503.0
5,106.0,201.0,290.0,52.0,118.0,94.0,343.0
6,369.0,55.0,14.0,80.0,74.0,44.0,112.0
7,25.0,NaN,NaN,5.0,NaN,3.0,15.0
8,NaN,156.0,7.0,NaN,2.0,NaN,NaN
9,17.0,NaN,1.0,9.0,NaN,3.0,13.0


# Work Tour Distance by Worker Type

In [18]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,12.238975,12.555871
1,Part-Time Worker,3.639760,6.248994
2,University Student,13.189476,5.321065
